In [ ]:
# !pip install transformers==4.8.2

# Grammar Error Corection

In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# model_name = 'deep-learning-analytics/GrammarCorrector'
model_name = 't5_gec_model_02' # model path
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:147: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  t = torch.tensor([], dtype=storage.dtype, device=storage._untyped_storage.device)


In [14]:
# input_text = "They can find new customers and introduce their services to each other. For instance, they commercials between the shows certify this issue.Furthermore TV is one of the tools that entertain people, although the target finds gift, so part of this shows for entertaining people."
input_text = "he are an teachers"
num_return_sequences = 3 # Số lượng câu generate
corrected_text = correct_grammar(input_text, num_return_sequences)
corrected_text

['he is a teacher.', 'he is an educator.', 'he is an instructor.']

In [12]:
corrected_text[0]

'he is a teacher.'

In [13]:
def mark_text(input_text, corrected_text):
    # Tách từng từ trong hai câu
    input_words = input_text.split()
    corrected_words = corrected_text.split()
    
    result = []

    # Duyệt qua từng từ trong hai câu
    for input_word, corrected_word in zip(input_words, corrected_words):
        # So sánh từ hiện tại trong hai câu
        if input_word == corrected_word:
            # Nếu từ giống nhau, thêm vào chuỗi kết quả với thẻ <span> bình thường
            result.append(f"<span>{input_word}</span>")
        else:
            # Nếu từ khác nhau, thêm vào chuỗi kết quả với thẻ <span> màu đỏ
            result.append(f"<span style='color: red;'>{corrected_word}</span>")

    # Kết hợp chuỗi kết quả thành một chuỗi duy nhất, thêm khoảng trắng giữa các thẻ <span>
    marked_text = ' '.join(result)

    return marked_text

marked_result = mark_text(input_text, corrected_text[0])
print(marked_result)


<span>he</span> <span style='color: red;'>is</span> <span style='color: red;'>a</span> <span style='color: red;'>teacher.</span>


# Question Generation and Fact - Check

In [1]:
# import fitz  # PyMuPDF
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sysme Hue\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In ord

In [21]:
import re
import textract

# Đọc nội dung của tài liệu PDF
text = textract.process('pdf/Report.pdf', encoding='utf-8')

# Sử dụng biểu thức chính quy để cắt thành các đoạn văn
all_paragraphs = re.split(r'\s{2,}', text.decode('utf-8'))
num_paragraph= len(text)
print(num_paragraph)

13876


In [22]:
list_para = []
list_para = [para for para in all_paragraphs if len(para.split()) >= 20] # list of paragraphs which have more than 20 words
len(list_para)

21

In [23]:
# Summarize paragraph using t5 model

summaries = []
for i,paragraph in enumerate(list_para):
    input_text = "summarize: " + paragraph
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)
    
    # if i==3: break

In [24]:
list_para[:4]

['Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
 "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this process of course recommendation for an upcoming semester, we have developed a system which deploys simple yet powerful recommendation techniques such as auto-encoders, hybrid matrix factorization and similarity based approaches. It is a GUI based system which takes an input of student's ID (which is stored in the backend d

In [25]:
# merge into result 

result = [{"paragraph": list_para[i], "summary": summaries[i]} for i in range(len(summaries))]

In [26]:
result[:5]

[{'paragraph': 'Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
  'summary': 'this paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters. this paper aims to develop a system that will help in recommendation of courses based on the performance of previous semesters.'},
 {'paragraph': "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this

In [87]:
# Clone this responsitory for questions generator

# !git clone https://github.com/amontgomerie/question_generator
# !pip install -r question_generator/requirements.txt -qq
!python run_qg.py --text_file question_generator/articles/twitter_hack.txt

python: can't open file 'c:\\Users\\Sysme Hue\\Desktop\\LMS\\Libs\\run_qg.py': [Errno 2] No such file or directory


In [90]:
%cd question_generator

c:\Users\Sysme Hue\Desktop\LMS\Libs\question_generator


In [92]:
# import file from another folder

# import sys
# # caution: path[0] is reserved for script path (or '' in REPL)
# sys.path.insert(1, 'question_generator')

from questiongenerator import QuestionGenerator


In [ ]:
qg = QuestionGenerator()

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Would remove:
    c:\users\sysme hue\appdata\local\programs\python\python310\lib\site-packages\transformers-4.30.2.dist-info\*
    c:\users\sysme hue\appdata\local\programs\python\python310\lib\site-packages\transformers\*
    c:\users\sysme hue\appdata\local\programs\python\python310\scripts\transformers-cli.exe
Proceed (Y/n)? 


In [36]:
text = result[2]["summary"]
q  = qg.generate(text, num_questions=3)

Generating questions...



c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluating QA pairs...



In [37]:
q

[{'question': 'how many students have taken the course?',
  'answer': 'the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.'},
 {'question': 'how many students have taken the iitt?',
  'answer': 'the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches.'},
 {'question': 'how many subjects do students have taken?',
  'answer': [{'answer': 'dataset', 'correct': False},
   {'answer': '7 Computer Science', 'correct': False},
   {'answer': '306', 'correct': True},
   {'answer': '739', 'correct': False}]}]

In [38]:
print(text)
for i in range(len(q)): print(q[i]["question"]) #  7 Computer Science passout batches.

the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches. the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
how many students have taken the course?
how many students have taken the iitt?
how many subjects do students have taken?


### Fact -check

In [11]:
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\tensorboard\__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [12]:
# BERT MODEL

class BERTClassificationModel(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BERTClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.cat((outputs.last_hidden_state[:, 0, :], outputs.last_hidden_state[:, -1, :]), dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [13]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

C:\Users\Sysme Hue\AppData\Local\Temp\ipykernel_10516\4253737859.py:2: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device = torch.device("cpu")


In [14]:
import torch
model_path = "model/bert_classification_model.pth"
# Tạo mô hình mới
loaded_model = BERTClassificationModel('bert-base-uncased', num_labels=3)


In [15]:
if torch.cuda.is_available():
    loaded_model.load_state_dict(torch.load(model_path))
else:
    loaded_model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

In [16]:
# Tải lại tokenizer từ đường dẫn đã lưu
loaded_tokenizer = BertTokenizer.from_pretrained("model/tokenizer")

In [30]:
max_seq_length = 128
def predict_premise_hypothesis(premise_text, hypothesis_text, model, tokenizer):
    # Chuẩn bị dữ liệu đầu vào cho mô hình
    inputs = tokenizer(premise_text, hypothesis_text, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return predicted_label

pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán

premise_text = "Yesterday, I bought a new computer"
hypothesis_text = "I did not buy a new computer"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])


premise_text : Yesterday, I bought a new computer 
 hypothesis_text : I did not buy a new computer
Predicted Label: contradiction


In [54]:
pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán
premise_text = "The likelihood is 100%"
hypothesis_text = "Maybe the probability is 100%"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

premise_text : The likelihood is 100% 
 hypothesis_text : Maybe the probability is 100%
Predicted Label: neutral


In [ ]:
premise_text = "The dataset had acquired for the student of 7 computer science"
hypothesis_text = result[2]["summary"]

predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)
print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

## Find similar Course

In [17]:
import pandas as pd
import numpy as np
import re
from mlxtend.frequent_patterns import apriori, association_rules
import en_core_web_sm
spc_en = en_core_web_sm.load()

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [18]:
# dataset --> https://www.kaggle.com/datasets/leewanhung/coursera-dataset?select=df_c.csv
course = pd.read_csv("../data/df_c.csv")
# id = pd.read_csv("../data/id.csv")

In [19]:
course_ids = [str(i).zfill(4) for i in range(1, len(course) + 1)]

# Thêm cột "Course_id" vào DataFrame
course.insert(0, 'Course_id', course_ids)

In [20]:
course.head()

,Course_id,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills,Specialized,Sequence,Vector,IDF,BERT_Encoded
0,0001,write feature length screenplay film television,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,write full length feature film script course w...,drama comedy peer screenwrite film document re...,Arts and Humanities,write feature length screenplay film televisio...,[3.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 2.0 0.0 0.0 0.0],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 3.09796989e-01 8.47294629e-02 4.00437862e-...
1,0002,business strategy business model canvas analys...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,end guide project fluent identifying create bu...,finance business plan persona user experience ...,Business,business strategy business model canvas analys...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-1.62371062e-02 -5.21499477e-02 3.00578535e-...
2,0003,silicon thin film solar cell,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,course consist general presentation solar cell...,chemistry physics solar energy film lambda cal...,Physical Science and Engineering,silicon thin film solar cell course consist ge...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 1.11733921e-01 2.74278939e-01 4.72732037e-...
3,0004,finance manager,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,come number always meet eye operational financ...,account receivable dupont analysis analysis ac...,Business,finance manager come number always meet eye op...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-4.35875319e-02 1.21336073e-01 3.25895369e-...
4,0005,retrieve datum use single table sql query,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,course learn effectively retrieve datum relati...,datum analysis select sql database management ...,Information Technology,retrieve datum use single table sql query cour...,[117.0 34.0 27.0 33.0 258.0 201.0 42.0 0.0 125...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 6.04763143e-02 3.37209553e-02 4.53004211e-...


In [21]:
# course.to_csv("../data/course.csv")

In [121]:
history = pd.read_csv("../data/history.csv", encoding='latin-1')

In [122]:
history # 395	user0396	

,User_id,History_course_id,History_course_name
0,user0001,"0687, 0355, 1677","Sequence Models, Natural Language Processing w..."
1,user0002,"0636, 0637, 1787","Exploratory Data Analysis, Exploratory Data An..."
2,user0003,"0120, 0286, 2712","Managing Big Data with MySQL, Introduction to ..."
3,user0004,"0406, 0603, 0840",Data Visualization and Communication with Tabl...
4,user0005,"0241, 2558, 3025, 3503",Deploy Models with TensorFlow Serving and Flas...
...,...,...,...
2133,user2134,"0685, 0819",NaN
2134,user2135,"1677, 2924, 1755, 2754",NaN
2135,user2136,"3361, 3369, 3399",NaN
2136,user2137,"0685, 0819",NaN


In [123]:
data = list(history['History_course_id'].apply(lambda x:x.split(",") ))

In [124]:
data

[['0687', ' 0355', ' 1677'],
 ['0636', ' 0637', ' 1787'],
 ['0120', ' 0286', ' 2712'],
 ['0406', ' 0603', ' 0840'],
 ['0241', ' 2558', ' 3025', ' 3503'],
 ['0450', ' 0662', ' 2696'],
 ['0391', ' 1808', ' 3454'],
 ['2533', ' 2543', ' 2564'],
 ['2919', ' 3097', ' 3260'],
 ['1581', ' 2154', ' 2725'],
 ['0088', ' 0112', ' 0233'],
 ['0211', ' 0220', ' 0241'],
 ['0149', ' 0211', ' 0288'],
 ['0220', ' 0241', ' 0288'],
 ['0199', ' 0211', ' 0241'],
 ['0120', ' 0286', ' 0413'],
 ['0066', ' 0149', ' 0396'],
 ['0108', ' 0279', ' 0687'],
 ['0330', ' 0376', ' 0530'],
 ['0058', ' 0221', ' 0849'],
 ['0485', ' 1210', ' 1792'],
 ['1366', ' 2486', ' 3352'],
 ['3399', ' 3423', ' 3454'],
 ['0543', ' 2854', ' 3352'],
 ['2845', ' 3349', ' 3361'],
 ['3352', ' 3369', ' 3399'],
 ['3334', ' 3352', ' 3361'],
 ['3361', ' 3369', ' 3370'],
 ['3361', ' 2708'],
 ['3352', ' 3369', ' 3376'],
 ['3352', ' 3369', ' 3423'],
 ['3454', ' 3465', ' 3472'],
 ['3465', ' 3472', ' 3475'],
 ['3475', ' 3483', ' 3491'],
 ['0636', ' 06

In [125]:
#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

,0005,0005,0014,0022,0024,0031,0044,0045,0053,0062,...,3465,3471,3475,3494,3498,3500,3502,3504,3514,Data analysis
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
df2 = df

In [127]:
#set a threshold value for the support value and calculate the support value.
apriori_t = apriori(df2, min_support = 0.01, use_colnames = True, verbose = 1)
apriori_t

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Processing 155 combinations | Sampling itemset size 54


,support,itemsets
0,0.011225,( 0014)
1,0.011225,( 0236)
2,0.014032,( 0257)
3,0.012629,( 0272)
4,0.013096,( 0364)
...,...,...
189,0.013096,"( 2993, 3108, 3469, 2842)"
190,0.013096,"( 2993, 3108, 3503, 2842)"
191,0.013096,"( 2993, 3503, 3469, 2842)"
192,0.013096,"( 3108, 3503, 3469, 2842)"


In [128]:
# Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(apriori_t, metric = "confidence", min_threshold = 0.6)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,( 0014),( 1289),0.011225,0.028064,0.011225,1.000000,35.633333,0.010910,inf,0.982971
1,( 0236),( 1115),0.011225,0.012161,0.011225,1.000000,82.230769,0.011089,inf,0.999054
2,( 1115),( 0236),0.012161,0.011225,0.011225,0.923077,82.230769,0.011089,12.854069,1.000000
3,( 0236),( 2542),0.011225,0.012161,0.011225,1.000000,82.230769,0.011089,inf,0.999054
4,( 2542),( 0236),0.012161,0.011225,0.011225,0.923077,82.230769,0.011089,12.854069,1.000000
...,...,...,...,...,...,...,...,...,...,...
514,"( 2993, 3108)","( 3469, 3503, 2842)",0.013096,0.013096,0.013096,1.000000,76.357143,0.012925,inf,1.000000
515,(2842),"( 2993, 3108, 3503, 3469)",0.013096,0.013096,0.013096,1.000000,76.357143,0.012925,inf,1.000000
516,( 3469),"( 2993, 3108, 3503, 2842)",0.017774,0.013096,0.013096,0.736842,56.263158,0.012864,3.750234,1.000000
517,( 2993),"( 3469, 3108, 3503, 2842)",0.013096,0.013096,0.013096,1.000000,76.357143,0.012925,inf,1.000000


In [129]:
df_ar[:30]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,( 0014),( 1289),0.011225,0.028064,0.011225,1.000000,35.633333,0.010910,inf,0.982971
1,( 0236),( 1115),0.011225,0.012161,0.011225,1.000000,82.230769,0.011089,inf,0.999054
2,( 1115),( 0236),0.012161,0.011225,0.011225,0.923077,82.230769,0.011089,12.854069,1.000000
3,( 0236),( 2542),0.011225,0.012161,0.011225,1.000000,82.230769,0.011089,inf,0.999054
4,( 2542),( 0236),0.012161,0.011225,0.011225,0.923077,82.230769,0.011089,12.854069,1.000000
5,( 0236),(0342),0.011225,0.013564,0.011225,1.000000,73.724138,0.011073,inf,0.997635
6,(0342),( 0236),0.013564,0.011225,0.011225,0.827586,73.724138,0.011073,5.734892,1.000000
7,( 0257),(0256),0.014032,0.013564,0.013564,0.966667,71.266667,0.013374,29.593078,1.000000
8,(0256),( 0257),0.013564,0.014032,0.013564,1.000000,71.266667,0.013374,inf,0.999526
9,( 0488),( 0272),0.012629,0.012629,0.012629,1.000000,79.185185,0.012469,inf,1.000000


In [131]:
result = course.loc[course['Course_id'] == "0992"]['Course Name'].iloc[0]
print('antecedents:', result)

antecedents: neural network deep learning


In [132]:
for n, antecedents in enumerate(df_ar['antecedents']):
    if isinstance(antecedents, frozenset):
        antecedents = list(antecedents)
    antecedent_ids = ','.join(map(str, antecedents)).strip()  # Chuyển thành danh sách và tách các ID

    antecedent_names = []
    for antecedent_id in antecedent_ids.split(','):
        antecedent_id = antecedent_id.strip()
        result = course.loc[course['Course_id'] == antecedent_id]['Course Name'].iloc[0]
        antecedent_names.append(result)

    antecedent_names_str = ', '.join(antecedent_names)

    print(n)
    print('antecedents:', antecedent_names_str)

    consequents = df_ar['consequents'][n]
    if isinstance(consequents, frozenset):
        consequents = list(consequents)
    consequent_ids = ','.join(map(str, consequents)).strip()  # Chuyển thành danh sách và tách các ID

    consequent_names = []
    for consequent_id in consequent_ids.split(','):
        consequent_id = consequent_id.strip()
        result = course.loc[course['Course_id'] == consequent_id]['Course Name'].iloc[0]
        consequent_names.append(result)

    consequent_names_str = ', '.join(consequent_names)

    print(consequent_names_str)
    print('consequents:', consequent_names_str, '\n', '-' * 30)


0
antecedents: business statistic analysis capstone
datum visualization python
consequents: datum visualization python 
 ------------------------------
1
antecedents: matrix method
precalculus periodic function
consequents: precalculus periodic function 
 ------------------------------
2
antecedents: precalculus periodic function
matrix method
consequents: matrix method 
 ------------------------------
3
antecedents: matrix method
logic economist
consequents: logic economist 
 ------------------------------
4
antecedents: logic economist
matrix method
consequents: matrix method 
 ------------------------------
5
antecedents: matrix method
game theory python
consequents: game theory python 
 ------------------------------
6
antecedents: game theory python
matrix method
consequents: matrix method 
 ------------------------------
7
antecedents: art music production
art music production
consequents: art music production 
 ------------------------------
8
antecedents: art music production
a

113
antecedents: business statistic analysis capstone
datum analytic accounting capstone
consequents: datum analytic accounting capstone 
 ------------------------------
114
antecedents: fake news detection machine learning
image compression k mean cluster
consequents: image compression k mean cluster 
 ------------------------------
115
antecedents: image compression k mean cluster
datum modeling regression analysis business
consequents: datum modeling regression analysis business 
 ------------------------------
116
antecedents: capstone autonomous runway detection iot
image compression k mean cluster
consequents: image compression k mean cluster 
 ------------------------------
117
antecedents: image compression k mean cluster
capstone autonomous runway detection iot
consequents: capstone autonomous runway detection iot 
 ------------------------------
118
antecedents: fake news detection machine learning
capstone autonomous runway detection iot
consequents: capstone autonomous runw

In [141]:
def recommend_courses(enrolled_courses, df_ar, num_recommendations=5):
    # Tạo một danh sách để lưu trữ các khoá học được đề xuất
    recommended_courses = []

    # Duyệt qua từng tập luật kết hợp trong df_ar
    for index, row in df_ar.iterrows():
        antecedents = row['antecedents']
        consequents = row['consequents']

        # Chuyển các ID trong antecedents thành các ID không có khoảng cách
        antecedents_cleaned = [course_id.replace(" ", "") for course_id in antecedents]

        # Kiểm tra nếu có ít nhất một khoá học từ antecedents có trong danh sách enrolled_courses
        if any(course_id in antecedents_cleaned for course_id in enrolled_courses):
            # Lấy danh sách các khoá học trong consequents
            recommended_courses.extend(consequents)

    # Loại bỏ các khoá học đã đăng ký và lặp lại
    recommended_courses = list(set(recommended_courses) - set(enrolled_courses))

    # Chọn một số lượng giới hạn của khoá học để đề xuất
    if len(recommended_courses) > num_recommendations:
        recommended_courses = recommended_courses[:num_recommendations]

    return recommended_courses

# Danh sách các khoá học mà người dùng đã đăng ký
enrolled_courses = ["0966", "1677", "1109"]
# ['1801', "0001","3356"]

# Đề xuất các khoá học dựa trên danh sách đã đăng ký và bảng df_ar
recommended_courses = recommend_courses(enrolled_courses, df_ar)
course_dict = dict(zip(course['Course_id'], course['Course Name']))
# In ra các khoá học được đề xuất
print("Enrolled Courses:")
for course_id in enrolled_courses:
    clean_course_id = course_id.replace(" ", "")  # Loại bỏ dấu cách
    course_name = course_dict.get(clean_course_id, 'Not Found')  # Lấy tên khoá học từ từ điển course_dict
    print(f"- {course_name} (ID: {clean_course_id})")
    
# In ra các khoá học được đề xuất, nhưng chỉ hiển thị những khoá học chưa được đăng ký
print("Recommend Courses:")
for course_id in recommended_courses:
    clean_course_id = course_id.replace(" ", "")  # Loại bỏ dấu cách
    course_name = course_dict.get(clean_course_id, 'Not Found')  # Lấy tên khoá học từ từ điển course_dict
    
    # Kiểm tra xem khoá học đã được đăng ký hay chưa
    if clean_course_id not in enrolled_courses:
        print(f"- {course_name} (ID: {clean_course_id})")


Enrolled Courses:
- interactive word embedding use word vec plotly (ID: 0966)
- nlp twitter sentiment analysis (ID: 1677)
- sentiment analysis deep learning use bert (ID: 1109)
Recommend Courses:
- transfer learn nlp tensorflow hub (ID: 2497)
